In [1]:
from dotenv import load_dotenv
import os

load_dotenv(override=True)

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
OPENAI_EMBEDDING_MODEL = os.getenv("OPENAI_EMBEDDING_MODEL")
PINECONE_INDEX_NAME = os.getenv("PINECONE_INDEX_NAME")
PINECONE_NAMESPACE = os.getenv("PINECONE_NAMESPACE")

In [ ]:
def load_latest_current_issues():
    """최신 현재이슈 JSON 파일을 자동으로 로드"""
    import glob
    import os
    
    json_files = glob.glob("data2/*_BigKinds_current_issues.json")
    if not json_files:
        raise FileNotFoundError("현재이슈 JSON 파일을 찾을 수 없습니다.")
    
    latest_file = max(json_files, key=os.path.getctime)
    print(f"📂 로드하는 파일: {latest_file}")
    
    with open(latest_file, "r", encoding="utf-8") as f:
        data = json.load(f)
    
    issues = data["issues"]
    docs = [
        Document(
            page_content=f"{item['제목']}\n{item['내용']}",
            metadata={"이슈번호": item["이슈번호"], "제목": item["제목"]}
        )
        for item in issues
    ]
    
    return docs

# 사용법
docs = load_latest_current_issues()

📂 로드하는 파일: data2\2025.07.18_16.01.32_BigKinds_current_issues.json


In [24]:

len(docs)   # csv 행이 1205개 있다는 것

10

In [25]:
# embedding 모델 객체 생성
from langchain_openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")


In [26]:
from pinecone import Pinecone, ServerlessSpec

# Pinecone 클라이언트를 초기화(객체생성)
pc = Pinecone(api_key=PINECONE_API_KEY)

In [27]:
# split하기
from langchain.text_splitter import RecursiveCharacterTextSplitter

# 텍스트 분할기 설정 (예: 1000자씩 분할)
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=450, 
    chunk_overlap=45,
    # length_function=tiktoken_len,  # 토큰 기반 길이 측정    
    length_function=len,  # 문자수   
    separators=["\n\n", "\n", " ", ""]
    )

# 문서를 분할
chunks = text_splitter.split_documents(docs)

In [28]:
len(chunks)

10

In [ ]:
# # vector sotre에 저장
# from langchain_pinecone import PineconeVectorStore

# batch_size = 30
#   # 한 번에 처리할 문서 수

# for i in range(0, len(chunks), batch_size):
#     batch_docs = chunks[i:i+batch_size]
    
#     if i == 0:
#         # 첫 번째 배치로 벡터 스토어 생성
#         vector_store = PineconeVectorStore.from_documents(
#             batch_docs,            # batch_size 수 만큼의 chunk
#             embedding=embeddings,  # 임베딩 벡터로 변환
#             index_name="lastproject",   # index 이름
#             namespace="past_issue"    
#         )
#     else:
#         # 이후 배치는 기존 벡터 스토어에 추가, # 내부적으로 임베딩 벡터로 변환
#         vector_store.add_documents(batch_docs)  
    
#     print(f"배치 {i//batch_size + 1} 완료: {len(batch_docs)}개 문서 업로드")

배치 1 완료: 28개 문서 업로드
